### Explore the competencies of teachers
Here we look at the competencies of teachers, which specific subjects are they trained in, do they have pedagogical comps. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sas7bdat
import seaborn as sns

import matplotlib as mpl

mpl.style.use('seaborn-whitegrid')  # Use 'seaborn-whitegrid' instead of 'ggplot'

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['axes.titleweight'] = 'bold'
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.prop_cycle'] = plt.cycler('color', plt.cm.Set1.colors)

plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.facecolor'] = '#f8f8f8'

pd.set_option('display.max_columns', None)

In [ ]:
#Get competency describtions with courses
filepath='../data/komp_koder.xlsx'
komp=pd.read_excel(filepath)

In [ ]:
#Get competencies
filepath='../data/macom_kompetencer.sas7bdat'
df =pd.read_sas(filepath, format='sas7bdat',encoding='iso-8859-1')

In [ ]:
#Map short description of comps to the actual meaning
map={
    'U':'Subject + Pedagogical',
    "F":'Subject',
    "P":'Pedagogical',
    "I":'No formal'
}

df['KompetenceNiveau']=df['KompetenceNiveau'].astype(str)


df['KompetenceNiveau']=df['KompetenceNiveau'].map(map)

In [ ]:
#Get the course for the competency codes.
komp=(komp.groupby('Kompetencekode',sort=False)
              .agg(**{'fag': ('Fag','first')})
              .reset_index()
              )

In [ ]:
# Get how many teachers per competency level
fun = (df.groupby('KompetenceNiveau', sort=False)
       .agg(**{'teachers': ('laerer_id', 'count')})
       .reset_index()
       ).sort_values('teachers', ascending=False)

x = fun['KompetenceNiveau']
y = fun['teachers']

fig, ax = plt.subplots(figsize=(16, 8))

# Create a bar plot to show the count of teachers for each competency level
ax.bar(x, y, color='steelblue')

ax.set_xlabel('Competency Level')
ax.set_ylabel('Count of teachers with specific competencies')
ax.set_title('Showing the different levels of competencies that teachers have')

plt.tight_layout()

# Save the data as a pickle file
fun.to_pickle('figures/teacher_comp.pkl')

# Save the figure as a PDF file
plt.savefig('figures/competency_levels.pdf')

plt.show()

In [ ]:
# Create a merge with competencies and code descriptions
new = pd.merge(df, komp, left_on='KompetenceKode', right_on='Kompetencekode', how='left')

# Group teachers by subject and count the number of teachers
fun = (new.groupby('fag', sort=False)
       .agg(**{'teachers': ('laerer_id', 'count')})
       .reset_index()
      )

# Select the top 10 subjects with the highest number of teachers
test = fun.sort_values('teachers', ascending=False).head(10)

# Convert the subject column to string type
test['fag'] = test['fag'].astype(str)

# Map English translations of subject descriptions
map = {
    'Dansk': 'Danish',
    'Engelsk': 'English',
    'Matematik': 'Mathematics',
    'Idræt': 'Physical Education',
    'KS - Samfundsfag': 'Social Studies',
    'Fysik': 'Physics',
    'Biologi': 'Biology',
    'Kemi': 'Chemistry',
    'Tysk begynder': 'Beginners German',
    'KS - Religion': 'Religion Studies'
}

test['fag'] = test['fag'].map(map)

fig, ax = plt.subplots(figsize=(16, 8))

x = test['fag']
y = test['teachers']
ax.bar(x, y, color='steelblue')

ax.set_xlabel('Subject')
ax.set_ylabel('Count of teachers with competencies in subjects')
ax.set_title('Subjects with Most Teachers Having Classified Competencies')

# Split x-tick labels at spaces for better readability
x_labels = [label.replace(' ', '\n') for label in x]
ax.set_xticklabels(x_labels)

plt.tight_layout()

# Save the data as a pickle file
test.to_pickle('figures/teacher_comp.pkl')

# Save the figure as a PDF file
plt.savefig('figures/teacher_comp.pdf')

plt.show()